# Pivot & Melt

## Kontingenční tabulka

Pivot table = kontingenční tabulka. 
Vytváření kontingenčních tabulek je proces ve kterém homogenní data agregujeme a jednotlivé "osy" agregace, 
resp. hodnoty v "osách" jsou určujícími pro řádky či sloupce.

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [97]:
sales = pd.read_csv(
    'sales_data.csv',
    parse_dates=['Date'])

In [98]:
sales.head()

,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
0,2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
1,2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2,2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
3,2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
4,2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418


In [99]:
import json
data = json.loads(sales.to_json(orient="table"))
print(data.keys())
data = data["data"]
print(len(data))
print(data[0])

dict_keys(['schema', 'data'])
113036
{'index': 0, 'Date': '2013-11-26T00:00:00.000', 'Day': 26, 'Month': 'November', 'Year': 2013, 'Customer_Age': 19, 'Age_Group': 'Youth (<25)', 'Customer_Gender': 'M', 'Country': 'Canada', 'State': 'British Columbia', 'Product_Category': 'Accessories', 'Sub_Category': 'Bike Racks', 'Product': 'Hitch Rack - 4-Bike', 'Order_Quantity': 8, 'Unit_Cost': 45, 'Unit_Price': 120, 'Profit': 590, 'Cost': 360, 'Revenue': 950}


Výběr hodnot v ose

### Příklad 1

**Příklad**

Znáte strukturu jednoho záznamu. Vytvořte kód, který zjistí všechny možné hodnoty v ose "Product_Category" (jaké hodnoty tento atribut může nabývat).

In [100]:
axis = "Product_Category"
product_category_values = [] # upravte s cílem splnit zadání
print(product_category_values)

[]


**Řešení**

In [101]:
axis = "Product_Category"
product_category_values = set(map(lambda item: item[axis], data))
print(product_category_values)

{'Accessories', 'Clothing', 'Bikes'}


### Příklad 2

**Příklad**

Vytvořte funkce vracející "True" / "False" podle příslušnosti do skupiny v ose "Product_Category"

In [102]:
product_category_selectors = dict()#zmente / naplnte, pouzijte vice radku

print(product_category_selectors)

row = data[0]
for key, value in product_category_selectors.items():
    print(key, value(row))

{}


**Řešení**

In [103]:
def createLambda(value):
    return lambda item: item["Product_Category"] == value

product_category_selectors = dict((value, createLambda(value)) for value in product_category_values)
print(product_category_selectors)

row = data[0]
for key, value in product_category_selectors.items():
    print(key, value(row))

{'Accessories': <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>, 'Clothing': <function createLambda.<locals>.<lambda> at 0x7f6b9a619a20>, 'Bikes': <function createLambda.<locals>.<lambda> at 0x7f6b9a619ab0>}
Accessories True
Clothing False
Bikes False


### Příklad 3

**Příklad**

Vytvořte funkci, jejímž výstupem je sada filtrů podle dikrétních hodnot u daného atributu (sloupce tabulky).
Jedná se o zobecnění předchozích úloh.

In [104]:
def createFilters(attributeName, table):
    def createLambda(value):
        return lambda item: item[attributeName] == value

    result = {}
    for row in table:
        #
        #
        #
        #
        pass
    return result

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)
print(ageFilters)
print(categoryFilters)

{}
{}


**Řešení**

In [105]:
def createFilters(attributeName, table):
    def createLambda(value):
        return lambda item: item[attributeName] == value

    result = {}
    for row in table:
        value = row[attributeName]
        if result.get(value, None) is None:
            result[value] = createLambda(value)
    return result

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)
print(ageFilters)
print(categoryFilters)

{'Youth (<25)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a170>, 'Adults (35-64)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a200>, 'Young Adults (25-34)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a290>, 'Seniors (64+)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a320>}
{'Accessories': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a440>, 'Clothing': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a4d0>, 'Bikes': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a560>}


### Příklad 4

**Příklad**

Kolik řádků v datech splňuje podmínku definovanou funkcí `product_category_selector`.

In [106]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = [] # Doplnte
print(len(filteredData))

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
0


**Řešení**

In [107]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = list(filter(product_category_selector, data))
print(len(filteredData))

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
70120


### Příklad 5

**Příklad**

Zjistěte součet attributu "Profit" pro řádky splňující podmínku definovanou funkcí.

In [108]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = list(filter(product_category_selector, data))
total = 0 # Doplnte
print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
0


**Řešení**

In [109]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = list(filter(product_category_selector, data))
mapped = map(lambda item: item["Profit"], filteredData)
total = sum(mapped)

print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
8862377


### Příklad 6

**Příklad**

Vytvořte funkci, která vztvoří funkci počítající sumu nad attributem pro řádky splňující podmínku.

In [110]:
def createSumarizer(attributeName, filterFunc):
    def result(data):
        filtered = [] #zmente
        mapped = [] #zmente
        total = sum(mapped)
        return total
    return result

[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)
sumarizer = createSumarizer("Profit", product_category_selector)
total = sumarizer(data)
print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
0


**Řešení**

In [111]:
def createSumarizer(attributeName, filterFunc):
    def result(data):
        filtered = filter(filterFunc, data)
        mapped = map(lambda item: item[attributeName], filtered)
        total = sum(mapped)
        return total
    return result

[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)
sumarizer = createSumarizer("Profit", product_category_selector)
total = sumarizer(data)
print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
8862377


### Příklad 7

**Příklad**

Ze seznamu filtrů vytvořte sumarizery a použijte výsledek na data

In [112]:
sumarizers = dict()#upravte

print(sumarizers)

print('-'*30)
for key, value in sumarizers.items():
    print(f"{key:10}\t{value(data)}")


{}
------------------------------


**Řešení**

In [113]:
sumarizers = dict((key, createSumarizer("Profit", filterfunc)) for key, filterfunc in product_category_selectors.items())
print(sumarizers)

print('-'*30)
for key, value in sumarizers.items():
    print(f"{key:10}\t{value(data)}")


{'Accessories': <function createSumarizer.<locals>.result at 0x7f6b9a61ad40>, 'Clothing': <function createSumarizer.<locals>.result at 0x7f6b9a61add0>, 'Bikes': <function createSumarizer.<locals>.result at 0x7f6b9a61ae60>}
------------------------------
Accessories	8862377
Clothing  	2839447
Bikes     	20519276


Výše uvedený výsledek je možné považovat za nejjednodušší kontingenční tabulku

### Příklad 8

**Příklad**

Seznamte se s funkcí `product` z knihovny `itertools`

In [114]:
import itertools

prod = list(itertools.product(['A', 'B'], ['1', '2']))
print(prod)

[('A', '1'), ('A', '2'), ('B', '1'), ('B', '2')]


### Příklad 9

**Příklad**

Použijte `product` na filtrech.

In [115]:
import itertools

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)

nestedNames = list(itertools.product(ageFilters.keys(), categoryFilters.keys()))
print(nestedNames)

[('Youth (<25)', 'Accessories'), ('Youth (<25)', 'Clothing'), ('Youth (<25)', 'Bikes'), ('Adults (35-64)', 'Accessories'), ('Adults (35-64)', 'Clothing'), ('Adults (35-64)', 'Bikes'), ('Young Adults (25-34)', 'Accessories'), ('Young Adults (25-34)', 'Clothing'), ('Young Adults (25-34)', 'Bikes'), ('Seniors (64+)', 'Accessories'), ('Seniors (64+)', 'Clothing'), ('Seniors (64+)', 'Bikes')]


In [116]:
import itertools

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)

nestedNames = list(itertools.product(ageFilters.items(), categoryFilters.items()))
print(nestedNames)

[(('Youth (<25)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b010>), ('Accessories', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b8e145ab0>)), (('Youth (<25)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b010>), ('Clothing', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b8e145990>)), (('Youth (<25)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b010>), ('Bikes', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b760>)), (('Adults (35-64)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a560>), ('Accessories', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b8e145ab0>)), (('Adults (35-64)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a560>), ('Clothing', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f

### Příklad 10

**Příklad**

Vytvořte vnořené filtry. Vnořený filtr vzniká jako `And` nad oběma filtry.

In [117]:
import itertools

def A(filterA, filterB):
    return lambda item: filterA(item) and filterB(item)

def A(*filters):
    def result_func(item):
        result = True
        for f in filters:
            if not (f(item)):
                result = False
                break
        return result
    return result_func

def createNestedFilters(data, attributeNameA, attributeNameB):
    filtersA = createFilters(attributeNameA, data)
    categoryFilters = createFilters(attributeNameB, data)
    nestedNames = list(itertools.product(filtersA.items(), categoryFilters.items()))
    
    result = {}
    for nestedName in nestedNames:
        filters = []
        names = []
        for name, function in nestedName:
            names.append(name)
            filters.append(function)
        level = result
        for name in names[:-1]:
            sublevel = level.get(name, None)
            if sublevel is None:
                sublevel = {}
                level[name] = sublevel
            level = sublevel
                
        masterfilter = A(*filters)
        name = names[-1]
        level[name] = masterfilter
    return result

nestedFilters = createNestedFilters(data, "Age_Group", "Product_Category")
print(nestedFilters)

{'Youth (<25)': {'Accessories': <function A.<locals>.result_func at 0x7f6b8e1cde10>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61ba30>, 'Bikes': <function A.<locals>.result_func at 0x7f6b9a61bac0>}, 'Adults (35-64)': {'Accessories': <function A.<locals>.result_func at 0x7f6b9a61bb50>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61bbe0>, 'Bikes': <function A.<locals>.result_func at 0x7f6b9a61bc70>}, 'Young Adults (25-34)': {'Accessories': <function A.<locals>.result_func at 0x7f6b9a61bd00>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61bd90>, 'Bikes': <function A.<locals>.result_func at 0x7f6b9a61be20>}, 'Seniors (64+)': {'Accessories': <function A.<locals>.result_func at 0x7f6b9a61beb0>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61bf40>, 'Bikes': <function A.<locals>.result_func at 0x7f6b841a0040>}}


In [118]:
def applyNestedFilters(filters, data):
    def inner(prefixes, filters, data):
        for key, f in filters.items():
            if isinstance(f, dict):
                for item in inner([*prefixes, key], f, data):
                    yield item
            else:
                yield [*prefixes, key], filter(f, data)
    return inner([], filters, data)
                
nestedFilters = createNestedFilters(data, "Age_Group", "Product_Category")
groupedData = list(applyNestedFilters(nestedFilters, data))
for prefixes, dataset in groupedData :
    #print(''.join(prefixes), sum(dataset))
    #print(prefixes, dataset)
    total = sum(map(lambda item: item["Profit"], dataset))
    print('\t'.join(prefixes), '\t', total)


Youth (<25)	Accessories 	 1255267
Youth (<25)	Clothing 	 410079
Youth (<25)	Bikes 	 2709246
Adults (35-64)	Accessories 	 4611837
Adults (35-64)	Clothing 	 1527164
Adults (35-64)	Bikes 	 10182581
Young Adults (25-34)	Accessories 	 2921240
Young Adults (25-34)	Clothing 	 884614
Young Adults (25-34)	Bikes 	 7580907
Seniors (64+)	Accessories 	 74033
Seniors (64+)	Clothing 	 17590
Seniors (64+)	Bikes 	 46542


Předchozí výstup je kontingenční tabulkou. Kontingenční tabulky je možno uspořádávat různě a také je možné mít v kontingenční tabulce více atributů (zde jsou dva).

In [119]:
import itertools

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)

nestedNames = list(itertools.product(ageFilters.keys(), categoryFilters.keys()))
print(nestedNames)

[('Youth (<25)', 'Accessories'), ('Youth (<25)', 'Clothing'), ('Youth (<25)', 'Bikes'), ('Adults (35-64)', 'Accessories'), ('Adults (35-64)', 'Clothing'), ('Adults (35-64)', 'Bikes'), ('Young Adults (25-34)', 'Accessories'), ('Young Adults (25-34)', 'Clothing'), ('Young Adults (25-34)', 'Bikes'), ('Seniors (64+)', 'Accessories'), ('Seniors (64+)', 'Clothing'), ('Seniors (64+)', 'Bikes')]


### Pandas

## "Dlouhá data"

https://towardsdatascience.com/reshape-pandas-dataframe-with-melt-in-python-tutorial-and-visualization-29ec1450bb02

In [120]:
data = [
    {'name': 'Anna', 'subject': 'Informatics', 'semester1': 'A', 'semester2': 'B', 'semester3': 'A'},
    {'name': 'Betty', 'subject': 'Informatics', 'semester1': 'C', 'semester2': 'A', 'semester3': 'A'},
    {'name': 'Cindy', 'subject': 'Mathematics', 'semester1': 'C', 'semester2': 'D', 'semester3': 'D'},
    {'name': 'Dita', 'subject': 'Biology', 'semester1': 'A', 'semester2': 'A', 'semester3': 'C'},
]

In [121]:
keys = {}
for row in data:
    for name in row.keys():
        keys[name] = True
        
for key in keys:
    print(f"{key:15}", end="")
print()
for row in data:
    for key in keys:
        value = row.get(key, None)
        print(f"{value:15}", end="")
    print()

name           subject        semester1      semester2      semester3      
Anna           Informatics    A              B              A              
Betty          Informatics    C              A              A              
Cindy          Mathematics    C              D              D              
Dita           Biology        A              A              C              


**Příklad**

Předpokládejte, že tabulka výše je kontingenční tabulkou. Vytvořte zdrojová data.

In [122]:
def ToLongForm(data, indexes=[], varName="variable", valueName="value"):
    for item in data:
        for key in item.keys():
            row = {}
            ################
            ################
            ################
            ################
            ################
            ################
            ################
            ################
            yield row
        
        
longData = list(ToLongForm(data, indexes=['name', 'subject'], varName='semester', valueName='grade'))
print(longData)

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]


**Řešení**

In [123]:
def ToLongForm(data, indexes=[], varName="variable", valueName="value"):
    for item in data:
        for key in item.keys():
            row = {}
            for i in indexes:
                row[i] = item.get(i, None)
                
            if key in indexes:
                continue
            row[varName] = key
            row[valueName] = item[key]
            yield row
        
longData = list(ToLongForm(data, indexes=['name', 'subject'], varName='semester', valueName='grade'))
print(longData)

[{'name': 'Anna', 'subject': 'Informatics', 'semester': 'semester1', 'grade': 'A'}, {'name': 'Anna', 'subject': 'Informatics', 'semester': 'semester2', 'grade': 'B'}, {'name': 'Anna', 'subject': 'Informatics', 'semester': 'semester3', 'grade': 'A'}, {'name': 'Betty', 'subject': 'Informatics', 'semester': 'semester1', 'grade': 'C'}, {'name': 'Betty', 'subject': 'Informatics', 'semester': 'semester2', 'grade': 'A'}, {'name': 'Betty', 'subject': 'Informatics', 'semester': 'semester3', 'grade': 'A'}, {'name': 'Cindy', 'subject': 'Mathematics', 'semester': 'semester1', 'grade': 'C'}, {'name': 'Cindy', 'subject': 'Mathematics', 'semester': 'semester2', 'grade': 'D'}, {'name': 'Cindy', 'subject': 'Mathematics', 'semester': 'semester3', 'grade': 'D'}, {'name': 'Dita', 'subject': 'Biology', 'semester': 'semester1', 'grade': 'A'}, {'name': 'Dita', 'subject': 'Biology', 'semester': 'semester2', 'grade': 'A'}, {'name': 'Dita', 'subject': 'Biology', 'semester': 'semester3', 'grade': 'C'}]
